In [ ]:
from google.colab import drive
import os
import numpy as np
import matplotlib.pyplot as plt
import torch


drive.mount('/content/gdrive')
folder ='gdrive/MyDrive/ProveMamba'
contents = os.listdir(folder)

##for el in contents:
 # if "txt" in el:
 #   file_path = os.path.join(folder, el)

 #   if os.path.exists(file_path):
 #     f = open(file_path, "r")
 #     f.readline()
 #     for row in f.readlines():
 #       print(row)

def load_data():
    Lista_Dati = []
    Lista_label = []
    Lista_Soggetto=[]
    file_list = os.listdir(".")
    for filename in contents:
        if "txt" in filename:
            file_path = os.path.join(folder, filename)
            f = open(file_path, "r")
            for row in f.readlines():
                Row_S = row.strip().split(",")
                #print(Row_S)
                # print(len(Row_S))
                Lista_Dati.append(
                    (float(Row_S[1]), float(Row_S[2]), float(Row_S[3]), float(Row_S[4]), float(Row_S[5]),
                     float(Row_S[6]),
                     float(Row_S[7]), float(Row_S[8]), float(Row_S[9]), float(Row_S[10]),
                     float(Row_S[11]), float(Row_S[12]), float(Row_S[13]), float(Row_S[14]), float(Row_S[15]),
                     float(Row_S[16]),
                     float(Row_S[17]), float(Row_S[18]), float(Row_S[19]),
                     float(Row_S[20]), float(Row_S[21]), float(Row_S[22]), float(Row_S[23]), float(Row_S[24])))
                Lista_label.append(Row_S[25])
                Lista_Soggetto.append(Row_S[26])

    return Lista_Dati, Lista_label, Lista_Soggetto

#load_data()

Lista_dati, Lista_label, Lista_Soggetto = load_data()
print(len(Lista_dati))


def Divido_per_soggetto(Lista_dati, Lista_label, Lista_Soggetto):
    Training =[]
    Test = []
    Label_Training = []
    Label_Test = []
    for i in range(len(Lista_Soggetto)):
        if Lista_Soggetto[i] == "AB06":
            Test.append(Lista_dati[i])
            Label_Test.append(Lista_label[i])
        else:
            Training.append(Lista_dati[i])
            Label_Training.append(Lista_label[i])

    return Training, Test, Label_Training, Label_Test


X,Y,x_label,y_label =Divido_per_soggetto(Lista_dati,Lista_label,Lista_Soggetto)


def finestre_temporali(Lista_dati, Lista_label):
    Grouped_List = []
    Lista_label_ridotta = []
    for i in range(0, len(Lista_dati), 100):
        group = Lista_dati[i:i + 100]
        group2 = Lista_label[i:i + 100]
        Grouped_List.append(group)
        Lista_label_ridotta.append(group2)
   # print(Grouped_List)
    #print(Lista_label_ridotta)
    Lista_indici = []
    Lista_dati_Corretta = []
    for i in range(len(Lista_label_ridotta)):
        #print(Lista_label_ridotta[i])
        if len(set(Lista_label_ridotta[i])) < 2:
            Lista_indici.append(set(Lista_label_ridotta[i]))
            Lista_dati_Corretta.append(Grouped_List[i])

    Lista_Indici_Finale = []
    for el in Lista_indici:
        lista = list(el)
        for el2 in lista:
            Lista_Indici_Finale.append(el2)

    return Lista_dati_Corretta, Lista_Indici_Finale


X2, X2_label =finestre_temporali(X, x_label)

Y2, Y2_label = finestre_temporali(Y, y_label)


## Senza Label ##
def do_label_numerica(X_1,y_1):
    X_new = []
    y_new = []
    for i in range(len(y_1)):
        if y_1[i] == "idle":
            continue
        else:
            X_new.append(X_1[i])
            y_new.append(y_1[i])

    return X_new, y_new


X2_new, X2_label_new = do_label_numerica(X2, X2_label)

Y2_new, Y2_label_new = do_label_numerica(Y2, Y2_label)


def do_label_Corretta(Nuove):
    Y_new2 = []
    for el in Nuove:
        if el =="walk-stairascent" or el =="stairascent" or el =="stairascent-walk":
            Y_new2.append(0)
        elif el == "walk-stairdescent" or el =="stairdescent" or el =="stairdescent-walk":
            Y_new2.append(1)
        elif el == "walk-rampascent" or el =="rampascent" or el =="rampascent-walk":
            Y_new2.append(2)
        elif el == "walk-rampdescent" or el =="rampdescent" or el =="rampdescent-walk":
            Y_new2.append(3)
        else:
            Y_new2.append(4)

    return Y_new2

Train_label = do_label_Corretta(X2_label_new)
Test_label = do_label_Corretta(Y2_label_new)

def creo_numpy_array(Y_new2C, X_new):
    New_X = []
    New_y = []
    for i in range(len(X_new)):
        if len(X_new[i]) < 100:
            continue
        else:
            New_X.append(X_new[i])
            New_y.append(Y_new2C[i])

    New_X = np.array(New_X)
    New_y = np.array(New_y)
    return New_X, New_y

Dati_X, label_x = creo_numpy_array(Train_label, X2_new)
Dati_test_y, label_test_y = creo_numpy_array(Test_label, Y2_new)
Dati_X_pytorch = torch.from_numpy(Dati_X).float()
label_x_pytorch = torch.from_numpy(label_x)
Dati_test_y_pytorch = torch.from_numpy(Dati_test_y)
label_test_y_pytorch = torch.from_numpy(label_test_y)

Mounted at /content/gdrive
347689


In [ ]:
from torch.utils.data import DataLoader, TensorDataset
dataset = TensorDataset(Dati_X_pytorch,label_x_pytorch)
train_loader = DataLoader(dataset, batch_size=1, shuffle=True)
dataset2 = TensorDataset(Dati_test_y_pytorch,label_test_y_pytorch)
test_loader = DataLoader(dataset2, batch_size=1,shuffle=True )